# Text Classification with Haystack: A Complete Tutorial

## Overview
This notebook demonstrates how to build an end-to-end text classification pipeline using Haystack. We'll create a system that:
1. Searches the web for articles about a specific topic
2. Fetches and processes the content
3. Automatically classifies articles into categories (Politics, Sport, Technology, Entertainment, Business)

## Learning Objectives
By the end of this tutorial, you will understand:
- How to build multi-component Haystack pipelines
- Web search and content fetching integration
- Zero-shot text classification using transformer models
- Creating custom Haystack components
- Chaining components together for complex workflows

## Prerequisites
- Basic Python knowledge
- Familiarity with NLP concepts
- Haystack framework installed
- SearchAPI key (stored in `.env` file)

In [ ]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.websearch import SearchApiWebSearch
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.routers import TransformersZeroShotTextRouter
from haystack import Pipeline
from haystack import component, Document
from haystack.utils import Secret
from typing import List

from dotenv import load_dotenv
import os
from pathlib import Path

# Load .env from the root of ch8 directory
root_dir = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd().parent
load_dotenv(root_dir / ".env")


True

## Step 1: Import Required Libraries

In this section, we import all the necessary components from Haystack:

- **Pipeline**: The core container that orchestrates component execution
- **DocumentCleaner**: Preprocesses text by removing extra whitespace and formatting
- **SearchApiWebSearch**: Performs web searches using the SearchAPI service
- **LinkContentFetcher**: Downloads content from URLs
- **HTMLToDocument**: Converts HTML content into Haystack Document objects
- **TransformersZeroShotTextRouter**: Classifies text without prior training on specific categories
- **component & Document**: Core building blocks for creating custom components
- **Secret**: Securely handles API keys from environment variables

We also use `python-dotenv` to load environment variables from a `.env` file.

In [2]:
text_router = TransformersZeroShotTextRouter(
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    labels=["Politics", "Sport","Technology","Entertainment", "Business"],)
text_router.warm_up()

@component
class NewsClassifier:
    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]) -> List[Document]:
        for document in documents:
            text = document.content
            meta = document.meta
            labels = text_router.run(text)
            meta['labels'] = list(labels.keys())[0]
            
        return {"documents": documents}

Device set to use mps


## Step 2: Create a Custom News Classifier Component

### Understanding Zero-Shot Classification
Zero-shot classification allows us to classify text into categories without training a model specifically on those categories. The model (`MoritzLaurer/deberta-v3-large-zeroshot-v2.0`) uses its general language understanding to determine which label best fits the text.

### The TransformersZeroShotTextRouter
We initialize the router with 5 predefined categories:
- **Politics**: Government, elections, policy news
- **Sport**: Athletics, competitions, teams
- **Technology**: Tech innovations, gadgets, software
- **Entertainment**: Movies, music, celebrities
- **Business**: Companies, markets, economy

The `warm_up()` method loads the model into memory for faster inference.

### Custom NewsClassifier Component
This component:
1. Takes a list of Document objects as input
2. Extracts the text content from each document
3. Runs classification on the text
4. Stores the predicted label in the document's metadata
5. Returns all documents with their classifications

**Key Design Pattern**: By storing classifications in metadata, we maintain the original document structure while enriching it with additional information.

## Step 3: Build the Haystack Pipeline

### Pipeline Architecture
Our pipeline consists of 5 connected components that process data sequentially:

```
SearchApiWebSearch → LinkContentFetcher → HTMLToDocument → DocumentCleaner → NewsClassifier
```

### Component Configuration

1. **SearchApiWebSearch**: 
   - Searches for articles (top_k=5 results)
   - Restricted to Britannica for quality content
   - Uses API key from environment variables

2. **LinkContentFetcher**: 
   - Downloads web pages from search results
   - 3 retry attempts for resilience
   - 10-second timeout to prevent hanging

3. **HTMLToDocument**: 
   - Converts raw HTML into structured Document objects
   - Extracts main content, ignoring navigation and ads

4. **DocumentCleaner**: 
   - Removes empty lines for cleaner text
   - Eliminates extra whitespaces
   - Removes specific substrings like '\n-'
   - Keeps text formatting consistent

5. **NewsClassifier**: 
   - Our custom component that classifies each document
   - Adds classification labels to metadata

### Pipeline Connections
Each `connect()` call defines data flow:
- `search.links` → `fetcher.urls`: URLs from search go to fetcher
- `fetcher` → `htmldocument`: Fetched HTML goes to converter
- `htmldocument` → `cleaner`: Documents go to cleaner
- `cleaner` → `classifier`: Clean documents get classified

In [3]:

# Initialize pipeline
pipeline = Pipeline()
web_search = SearchApiWebSearch(top_k=5,
                                api_key=Secret.from_env_var("SEARCH_API_KEY"),
                                allowed_domains=["https://www.britannica.com/"])
link_content = LinkContentFetcher(retry_attempts=3,
                                  timeout=10)
html_to_doc = HTMLToDocument()
document_cleaner = DocumentCleaner(
                                remove_empty_lines=True,
                                remove_extra_whitespaces=True,
                                remove_repeated_substrings=False,
                                remove_substrings=['\n-']
                            )


# Add components
pipeline.add_component(name='search', instance=web_search)
pipeline.add_component(name ='fetcher' , instance= link_content)
pipeline.add_component(name='htmldocument', instance=html_to_doc)
pipeline.add_component(name='cleaner', instance=document_cleaner)
pipeline.add_component(name='classifier', instance=NewsClassifier())

# Connect components to one another
pipeline.connect("search.links", "fetcher.urls")
pipeline.connect("fetcher", "htmldocument")
pipeline.connect("htmldocument", "cleaner")
pipeline.connect("cleaner", "classifier")



🚅 Components
  - search: SearchApiWebSearch
  - fetcher: LinkContentFetcher
  - htmldocument: HTMLToDocument
  - cleaner: DocumentCleaner
  - classifier: NewsClassifier
🛤️ Connections
  - search.links -> fetcher.urls (list[str])
  - fetcher.streams -> htmldocument.sources (list[ByteStream])
  - htmldocument.documents -> cleaner.documents (list[Document])
  - cleaner.documents -> classifier.documents (list[Document])

## Step 4: Run the Pipeline

### Executing the Search and Classification
Now we'll search for articles about "Elon Musk" and automatically classify them.

**How it works:**
1. The query is passed to the search component
2. Search finds relevant URLs from Britannica
3. Content is fetched, converted, and cleaned
4. Each article is classified into one of our 5 categories
5. Results are returned with classifications in metadata

**Expected Output:**
Articles about Elon Musk might be classified as:
- **Technology**: Articles about Tesla, SpaceX innovations
- **Business**: Articles about his companies and ventures
- **Politics**: Articles about policy positions or government interactions

In [4]:
query = "Elon Musk"
output = pipeline.run(data={"search":{"query":query}})

In [5]:
extracted_documents = output['classifier']['documents']

## Step 5: Extract and Analyze Results

### Accessing Classified Documents
The pipeline output is a dictionary with nested results. We extract the documents from:
- `output['classifier']['documents']` - documents processed by our classifier component

Each document contains:
- **content**: The full text of the article
- **meta**: Metadata including:
  - Original URL
  - Title
  - **labels**: Our classification result!

In [6]:
for i in range(len(extracted_documents)):
    print(extracted_documents[i].meta)
    # to get content of the document use extracted_documents[i].content
    print("----")

{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/Elon-Musk', 'labels': 'Technology'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/Zip2', 'labels': 'Technology'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/topic/Department-of-Government-Efficiency-United-States', 'labels': 'Politics'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/Tesla-Motors', 'labels': 'Business'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/xAI', 'labels': 'Technology'}
----


## Step 6: Display Classification Results

### Inspecting Document Metadata
This loop displays the metadata for each classified document, including:
- Source URL
- Classification label
- Any other metadata collected during processing

**Try This:**
- Uncomment `extracted_documents[i].content` to see the full article text
- Notice how articles about the same person can be classified differently based on their focus
- Try different queries to see how the classifier performs on various topics

### Understanding the Results
The classification is based on the overall content and context of each article. The zero-shot model uses its understanding of language patterns to determine which category best fits, even though it was never explicitly trained on these specific news categories.

## Exercises and Extensions

### Try These Experiments:

1. **Different Queries**: 
   - Search for "climate change", "Olympics", or "cryptocurrency"
   - Observe how classification changes with topic

2. **Modified Categories**:
   - Change the labels in `TransformersZeroShotTextRouter` to different categories
   - Example: ["Science", "Health", "Education", "Environment"]

3. **Enhanced Classification**:
   - Add confidence scores by accessing `labels` dictionary values
   - Filter documents by classification (e.g., only show Technology articles)

4. **Data Analysis**:
   ```python
   # Count articles per category
   from collections import Counter
   categories = [doc.meta['labels'] for doc in extracted_documents]
   print(Counter(categories))
   ```

5. **Multiple Queries**:
   - Run the pipeline with multiple queries in a loop
   - Compare classification distributions across different topics

### Key Takeaways:
- ✅ Haystack pipelines enable modular, reusable NLP workflows
- ✅ Zero-shot classification works without domain-specific training
- ✅ Custom components extend Haystack's functionality
- ✅ Component chaining creates powerful end-to-end systems
- ✅ Metadata enrichment preserves document structure while adding intelligence

### Next Steps:
- Explore other Haystack components (embedders, retrievers, generators)
- Build classification systems for your own domains
- Combine classification with other NLP tasks (summarization, Q&A)
- Deploy pipelines as REST APIs using Hayhooks